In [1]:
import pandas as pd
import utils
import preprocessing
import factors
from factors import *
import numpy as np
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Chiara\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Chiara\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def load_data(year):
    #import BDD file
    df_y1_y2 = utils.import_BDD(f"data/BDD{year}.csv")
    #import qualtrics file (substituted by an empty file if not avaialable)
    df_y3 = utils.import_qualtrics(f"data/qualtrics{year}.csv")
    #merge the two data sources (BDD and qualtrics)
    df_all = pd.merge(df_y1_y2,df_y3, how="outer", on="bid")
    #import file containing admission codes
    admissions = pd.read_csv("data/admission.csv")
    admissions.dropna(inplace = True)
    admissions.drop("STVATTS_DESC", axis = 1, inplace = True)

    #join the admissions codes with the general df
    df = df_all.merge(admissions, how = "left", left_on = "admission1", right_on = "STVATTS_CODE")
    df.drop("STVATTS_CODE", axis = 1, inplace = True)
    return df

In [3]:
def analysis_to_excel(df_factors, parameters_df, file_name):
    parameters_df = parameters_df.T
    parameters_df.columns = ["Parameters", "Choice"]
    
    with pd.ExcelWriter(file_name) as writer:
        parameters_df.to_excel(writer, sheet_name = "parameters_used", index = False)
        df_factors.to_excel(writer, sheet_name = "factors", index = False)
        for group in groups:
            temp = factors.score(df_factors, group, weights, na_method, weighted=False)
            temp.to_excel(writer, sheet_name=group, index=False)

In [4]:
#define a function that filters the dataset based on the new criteria given:
#1. First salary avaialable
#2. At least of the two subsequent salaries available
def new_filter(df):
    df_new = df

    #1. Keep just rows for which we have the salary at year 1
    df_new = df_new[df_new.salary_y1.notna()]

    #2. Keep just the rows for which we have at least one of the two subsequent years
    df_new = df_new[(df_new.salary_y2.notna())|(df_new.salary_y3.notna())]

    print(f"Number of initial observations: {len(df)}")
    print(f"Number of observations after applying the filter: {len(df_new)}")

    return df_new

In [5]:
#set parameters

# PREPROCESSING
value_binary = 2
method_range = "mean"

# FACTORS
use_recommendations = True
years_back = 0

# WEIGHT FOR EACH FACTOR (for final score)
weights = {
        "is_woman":5,
        "is_int":5,
        "career_jump":5,
        "satisfaction":5,
        "career_service":5,
        "mobility":8,
        "salary":20,
        "salary_increase_perc":5,
        "salary_increase_abs":5
    }

# HANDLING OF MISSING VALUES
#substitute with group mean
na_method = "group"

# HANDLING OF OUTLIERS
out_salary = (1-0.95)/2 
out_salary_increase= (1-0.95)/2
outliers_method = "substitute"
#Note: quantiles are computed as follows in the function that eliminates outliers
    #q_low = new_df["salary"].quantile(1-q)
    #q_hi  = new_df["salary"].quantile(q)



# 2018

## Analysis no filter

In [6]:
qualtrics_data = True

#load data for 2018
df18 = load_data(2018)

#do all the necessary prepocessing to get the columns from which the different factors are computed
df18 = preprocessing.preprocessing_df(df18, method_range, value_binary)

In [7]:
#run the analysis 
#compute all the single variables
df_factors_18 = factors.factors_df(df18, 
                                grouping_criteria=["Admission", "Admission AST"],
                                years_before = years_back, qualtrics = qualtrics_data,
                                recommendations = use_recommendations,
                                q = out_salary,
                                q_increase = out_salary_increase,
                                outliers = outliers_method)

In [8]:
#add info about the chaires (not for 2018)
#chaires18 = pd.read_csv(f"data/chaires2018.csv")
#chaires18["Chaires"] = chaires18.Chaires.apply(lambda x: x.split(",")[0] if type(x)==str else x)
#df_factors_18 = pd.merge(df_factors_18, chaires18, how = "left", left_on= "BID", right_on="Ecole_BID")
#df_factors_18.drop("Ecole_BID", axis = 1)

#groups = ["is_woman", "is_int", "Admission", "Admission AST", "Chaires"]
groups = ["is_woman", "is_int", "Admission", "Admission AST"]

parameters = pd.DataFrame([["Qualtrics","method_range",
                                    "use_recommendations","value_recommendations",
                                    "Nb of years back","Method missing values",
                                    "Quantile salary","Quantile salary increase",
                                    "Method outliers"],
                            [qualtrics_data,
                            method_range,
                            use_recommendations,
                            value_binary,
                            years_back,
                            na_method,
                            1-out_salary*2,
                            1-out_salary_increase*2,
                            outliers_method]])

#save analysis to excel
analysis_to_excel(df_factors_18,parameters, file_name = "analysis/analysis2018.xlsx")

In [9]:
#visualize results for Admission
pd.read_excel("analysis/analysis2018.xlsx", sheet_name = "Admission AST")


,Group: Admission AST,is_woman (5),is_int (5),career_jump (5),satisfaction (5),career_service (5),mobility (8),salary (20),salary_increase_perc (5),salary_increase_abs (5),total_score,count,missing_salary_count
0,ASC,0.986595,0.075067,0.543103,0.763966,0.559671,0.200565,0.241249,0.038816,0.239690,22.464038,373,191
1,AST,0.967213,0.475410,0.527919,0.779738,0.551020,0.352025,0.243377,0.044437,0.246011,25.642489,366,194
2,DD,0.888889,0.555556,0.538462,0.800000,0.750000,0.531250,0.287176,0.029766,0.176906,28.691418,36,25
3,AST (ASTI+ASTF) + DD,0.960199,0.482587,0.528571,0.781250,0.558824,0.368272,0.246010,0.043484,0.241404,25.847976,402,219
4,Other,0.000000,0.000000,1.000000,0.750000,0.000000,0.000000,0.000000,0.089956,0.323183,10.815698,2,1
5,General,0.975547,0.285714,0.537246,0.771298,0.557233,0.283498,0.242970,0.041222,0.240799,24.172687,777,411


## New analysis applying filter

In [10]:
df18_new = new_filter(df18)

#run the analysis of the filtered dataset
#compute all the single variables
df_factors_18 = factors.factors_df(df18_new, 
                                grouping_criteria=["Admission", "Admission AST"],
                                years_before = years_back, qualtrics = qualtrics_data,
                                recommendations = use_recommendations,
                                q = out_salary,
                                q_increase = out_salary_increase,
                                outliers = outliers_method)

#add info about the chaires (not for 2018)
#chaires18 = pd.read_csv(f"data/chaires2018.csv")
#chaires18["Chaires"] = chaires18.Chaires.apply(lambda x: x.split(",")[0] if type(x)==str else x)
#df_factors_18 = pd.merge(df_factors_18, chaires18, how = "left", left_on= "BID", right_on="Ecole_BID")
#df_factors_18.drop("Ecole_BID", axis = 1)

#groups = ["is_woman", "is_int", "Admission", "Admission AST", "Chaires"]
groups = ["is_woman", "is_int", "Admission", "Admission AST"]

parameters = pd.DataFrame([["Qualtrics","method_range",
                                    "use_recommendations","value_recommendations",
                                    "Nb of years back","Method missing values",
                                    "Quantile salary","Quantile salary increase",
                                    "Method outliers"],
                            [qualtrics_data,
                            method_range,
                            use_recommendations,
                            value_binary,
                            years_back,
                            na_method,
                            1-out_salary*2,
                            1-out_salary_increase*2,
                            outliers_method]])

#save analysis to excel
analysis_to_excel(df_factors_18,parameters, file_name = "analysis/new_analysis2018.xlsx")

Number of initial observations: 777
Number of observations after applying the filter: 204


In [11]:
#visualize results for Admission
pd.read_excel("analysis/new_analysis2018.xlsx", sheet_name = "Admission AST")

,Group: Admission AST,is_woman (5),is_int (5),career_jump (5),satisfaction (5),career_service (5),mobility (8),salary (20),salary_increase_perc (5),salary_increase_abs (5),total_score,count,missing_salary_count
0,ASC,0.944000,0.064000,0.520325,0.784500,0.562667,0.209677,0.275095,0.032845,0.231169,22.876846,125,36
1,AST,0.613333,0.400000,0.449275,0.793333,0.577778,0.283784,0.284268,0.048256,0.215332,23.442178,75,22
2,DD,0.666667,0.666667,0.333333,0.916667,0.666667,0.666667,0.048565,0.017541,0.053508,22.909883,3,0
3,AST (ASTI+ASTF) + DD,0.615385,0.410256,0.444444,0.798077,0.581197,0.298701,0.271641,0.046639,0.206342,23.334141,78,22
4,Other,0.000000,0.000000,1.000000,0.750000,0.000000,0.000000,0.000000,0.084723,0.279965,10.573438,1,0
5,General,0.892157,0.196078,0.494898,0.789522,0.566993,0.242574,0.271886,0.038396,0.222198,23.379526,204,58


# 2019

## Analysis no filter

In [12]:
qualtrics_data = True

#load data for 2018
df19 = load_data(2019)

#do all the necessary prepocessing to get the columns from which the different factors are computed
df19 = preprocessing.preprocessing_df(df19, method_range, value_binary)

#run the analysis
#compute all the single variables
df_factors_19 = factors.factors_df(df19, 
                                grouping_criteria=["Admission", "Admission AST"],
                                years_before = years_back, qualtrics = qualtrics_data,
                                recommendations = use_recommendations,
                                q = out_salary,
                                q_increase = out_salary_increase,
                                outliers = outliers_method)


#add info about the chaires
chaires19 = pd.read_csv(f"data/chaires2019.csv")
chaires19["Chaires"] = chaires19.Chaires.apply(lambda x: x.split(",")[0] if type(x)==str else x)
df_factors_19 = pd.merge(df_factors_19, chaires19, how = "left", left_on= "BID", right_on="Ecole_BID")
df_factors_19.drop("Ecole_BID", axis = 1)

groups = ["is_woman", "is_int", "Admission", "Admission AST", "Chaires"]

parameters = pd.DataFrame([["Qualtrics","method_range",
                                    "use_recommendations","value_recommendations",
                                    "Nb of years back","Method missing values",
                                    "Quantile salary","Quantile salary increase",
                                    "Method outliers"],
                            [qualtrics_data,
                            method_range,
                            use_recommendations,
                            value_binary,
                            years_back,
                            na_method,
                            1-out_salary*2,
                            1-out_salary_increase*2,
                            outliers_method]])

#save analysis to excel
analysis_to_excel(df_factors_19,parameters, file_name = "analysis/analysis2019.xlsx")

In [13]:
#visualize results for Admission
pd.read_excel("analysis/analysis2019.xlsx", sheet_name = "Admission AST")

,Group: Admission AST,is_woman (5),is_int (5),career_jump (5),satisfaction (5),career_service (5),mobility (8),salary (20),salary_increase_perc (5),salary_increase_abs (5),total_score,count,missing_salary_count
0,ASC,0.989474,0.087071,0.224599,0.722642,0.523447,0.109756,0.225806,0.216620,0.523890,21.832893,380,318
1,AST,0.969388,0.540816,0.215278,0.780405,0.559091,0.164179,0.221328,0.236725,0.485749,24.677251,392,343
2,DD,0.722222,0.777778,0.277778,0.840909,0.651515,0.500000,0.292958,0.226978,0.542158,30.055842,36,31
3,AST (ASTI+ASTF) + DD,0.995327,0.560748,0.222222,0.785861,0.567493,0.200000,0.227960,0.235913,0.491037,25.452209,428,374
4,Other,0.400000,0.000000,0.000000,0.750000,0.400000,0.500000,0.000000,0.202850,0.470707,15.117786,5,4
5,General,0.988930,0.336207,0.222222,0.752918,0.543137,0.157233,0.224871,0.225130,0.509357,23.644782,813,696


In [14]:
#visualize results for Chair
chaires = pd.read_excel("analysis/analysis2019.xlsx", sheet_name = "Chaires")

chaires[(chaires["Group: Chaires"]=="Accenture Bus Analytics Chair") |(chaires["Group: Chaires"]=="General")]

,Group: Chaires,is_woman (5),is_int (5),career_jump (5),satisfaction (5),career_service (5),mobility (8),salary (20),salary_increase_perc (5),salary_increase_abs (5),total_score,count,missing_salary_count
8,Accenture Bus Analytics Chair,1.00000,0.380000,0.266667,0.750000,0.531532,0.142857,0.150596,0.231808,0.555101,22.730313,50,37
28,General,0.98893,0.336207,0.222222,0.752918,0.543137,0.157233,0.224871,0.225130,0.509357,23.644782,813,696


## New analysis applying filter

In [15]:
df19_new = new_filter(df19)

#run the analysis of the filtered dataset
#compute all the single variables
df_factors_19 = factors.factors_df(df19_new, 
                                grouping_criteria=["Admission", "Admission AST"],
                                years_before = years_back, qualtrics = qualtrics_data,
                                recommendations = use_recommendations,
                                q = out_salary,
                                q_increase = out_salary_increase,
                                outliers = outliers_method)

#add info about the chaires
chaires19 = pd.read_csv(f"data/chaires2019.csv")
chaires19["Chaires"] = chaires19.Chaires.apply(lambda x: x.split(",")[0] if type(x)==str else x)
df_factors_19 = pd.merge(df_factors_19, chaires19, how = "left", left_on= "BID", right_on="Ecole_BID")
df_factors_19.drop("Ecole_BID", axis = 1)

groups = ["is_woman", "is_int", "Admission", "Admission AST", "Chaires"]

parameters = pd.DataFrame([["Qualtrics","method_range",
                                    "use_recommendations","value_recommendations",
                                    "Nb of years back","Method missing values",
                                    "Quantile salary","Quantile salary increase",
                                    "Method outliers"],
                            [qualtrics_data,
                            method_range,
                            use_recommendations,
                            value_binary,
                            years_back,
                            na_method,
                            1-out_salary*2,
                            1-out_salary_increase*2,
                            outliers_method]])

#save analysis to excel
analysis_to_excel(df_factors_19,parameters, file_name = "analysis/new_analysis2019.xlsx")

Number of initial observations: 813
Number of observations after applying the filter: 170


In [16]:
#visualize results for Admission
pd.read_excel("analysis/new_analysis2019.xlsx", sheet_name = "Admission AST")

,Group: Admission AST,is_woman (5),is_int (5),career_jump (5),satisfaction (5),career_service (5),mobility (8),salary (20),salary_increase_perc (5),salary_increase_abs (5),total_score,count,missing_salary_count
0,ASC,1.000000,0.040816,0.322222,0.790816,0.500000,0.088889,0.210506,0.211143,0.529787,21.895159,98,61
1,AST,0.939394,0.454545,0.366667,0.821970,0.641414,0.093750,0.227780,0.231037,0.468035,24.920921,66,37
2,DD,0.666667,1.000000,0.333333,0.916667,0.777778,1.000000,0.154930,0.192834,0.473904,32.904505,3,2
3,AST (ASTI+ASTF) + DD,0.956522,0.478261,0.365079,0.826087,0.647343,0.121212,0.225352,0.229764,0.468261,25.333322,69,39
4,Other,0.000000,0.000000,0.000000,0.750000,0.444444,0.000000,0.000000,0.202850,0.490526,9.439105,3,2
5,General,1.000000,0.217647,0.335484,0.804412,0.558824,0.101266,0.213960,0.219236,0.504178,23.288234,170,102


In [20]:
#visualize results for Chair
chaires = pd.read_excel("analysis/new_analysis2019.xlsx", sheet_name = "Chaires")

chaires[(chaires["Group: Chaires"]=="Accenture Bus Analytics Chair") |(chaires["Group: Chaires"]=="General")]

,Group: Chaires,is_woman (5),is_int (5),career_jump (5),satisfaction (5),career_service (5),mobility (8),salary (20),salary_increase_perc (5),salary_increase_abs (5),total_score,count,missing_salary_count
3,Accenture Bus Analytics Chair,0.909091,0.272727,0.454545,0.772727,0.606061,0.000000,0.110664,0.219218,0.549699,21.133623,11,4
20,General,1.000000,0.217647,0.335484,0.804412,0.558824,0.101266,0.213960,0.219236,0.504178,23.288234,170,102


# 2020

## Analysis no filter

In [24]:
qualtrics_data = False

#no recommendations for 2020
use_recommendations = False

#last year is not available so we need to go back of at least one year
years_back = 1

#load data for 2020
df20 = load_data(2020)

#do all the necessary prepocessing to get the columns from which the different factors are computed
df20 = preprocessing.preprocessing_df(df20, method_range, value_binary)

#run the analysis
#compute all the single variables
df_factors_20 = factors.factors_df(df20, 
                                grouping_criteria=["Admission", "Admission AST"],
                                years_before = years_back, qualtrics = qualtrics_data,
                                recommendations = use_recommendations,
                                q = out_salary,
                                q_increase = out_salary_increase,
                                outliers = outliers_method)


#add info about the chaires
chaires20 = pd.read_csv(f"data/chaires2020.csv")
chaires20["Chaires"] = chaires20.Chaires.apply(lambda x: x.split(",")[0] if type(x)==str else x)
df_factors_20 = pd.merge(df_factors_20, chaires20, how = "left", left_on= "BID", right_on="Ecole_BID")
df_factors_20.drop("Ecole_BID", axis = 1)

groups = ["is_woman", "is_int", "Admission", "Admission AST", "Chaires"]

parameters = pd.DataFrame([["Qualtrics","method_range",
                                    "use_recommendations","value_recommendations",
                                    "Nb of years back","Method missing values",
                                    "Quantile salary","Quantile salary increase",
                                    "Method outliers"],
                            [qualtrics_data,
                            method_range,
                            use_recommendations,
                            value_binary,
                            years_back,
                            na_method,
                            1-out_salary*2,
                            1-out_salary_increase*2,
                            outliers_method]])

#save analysis to excel
analysis_to_excel(df_factors_20,parameters, file_name = "analysis/analysis2020.xlsx")

In [25]:
#visualize results for Admission
pd.read_excel("analysis/analysis2020.xlsx", sheet_name = "Admission AST")

,Group: Admission AST,is_woman (5),is_int (5),career_jump (5),satisfaction (5),career_service (5),mobility (8),salary (20),salary_increase_perc (5),salary_increase_abs (5),total_score,count,missing_salary_count
0,ASC,0.930070,0.083969,0.076433,0.699468,0.449005,0.995327,0.194679,0.193385,0.154855,24.792119,286,152
1,AST,0.872000,0.415966,0.059701,0.765121,0.545710,0.873684,0.222226,0.216120,0.171945,26.666814,250,152
2,DD,1.000000,0.652174,0.000000,0.781250,0.444444,0.619048,0.311860,0.261246,0.212310,27.946696,24,18
3,AST (ASTI+ASTF) + DD,0.883212,0.436782,0.052632,0.766544,0.537468,0.848341,0.227397,0.219881,0.174828,26.691400,274,170
4,Other,0.000000,0.000000,NaN,0.750000,0.166667,1.000000,NaN,NaN,NaN,NaN,2,2
5,General,0.982206,0.259048,0.064725,0.732464,0.491162,0.922535,0.208976,0.205614,0.164302,26.057399,562,324


In [29]:
#visualize results for Chair
chaires = pd.read_excel("analysis/analysis2020.xlsx", sheet_name = "Chaires")

chaires[(chaires["Group: Chaires"]=="CH ACCENTURE") |(chaires["Group: Chaires"]=="General")]

,Group: Chaires,is_woman (5),is_int (5),career_jump (5),satisfaction (5),career_service (5),mobility (8),salary (20),salary_increase_perc (5),salary_increase_abs (5),total_score,count,missing_salary_count
11,CH ACCENTURE,0.909091,0.290323,0.058824,0.750000,0.517241,0.846154,0.280806,0.170337,0.118877,26.458802,33,23
28,General,0.982206,0.259048,0.064725,0.732464,0.491162,0.922535,0.208976,0.205614,0.164302,26.057399,562,324


## New Analysis with filter


In [30]:
df20_new = new_filter(df20)

#run the analysis of the filtered dataset
#compute all the single variables
df_factors_20 = factors.factors_df(df20_new, 
                                grouping_criteria=["Admission", "Admission AST"],
                                years_before = years_back, qualtrics = qualtrics_data,
                                recommendations = use_recommendations,
                                q = out_salary,
                                q_increase = out_salary_increase,
                                outliers = outliers_method)

#add info about the chaires
chaires20 = pd.read_csv(f"data/chaires2020.csv")
chaires20["Chaires"] = chaires20.Chaires.apply(lambda x: x.split(",")[0] if type(x)==str else x)
df_factors_20 = pd.merge(df_factors_20, chaires20, how = "left", left_on= "BID", right_on="Ecole_BID")
df_factors_20.drop("Ecole_BID", axis = 1)

groups = ["is_woman", "is_int", "Admission", "Admission AST", "Chaires"]

parameters = pd.DataFrame([["Qualtrics","method_range",
                                    "use_recommendations","value_recommendations",
                                    "Nb of years back","Method missing values",
                                    "Quantile salary","Quantile salary increase",
                                    "Method outliers"],
                            [qualtrics_data,
                            method_range,
                            use_recommendations,
                            value_binary,
                            years_back,
                            na_method,
                            1-out_salary*2,
                            1-out_salary_increase*2,
                            outliers_method]])

#save analysis to excel
analysis_to_excel(df_factors_20,parameters, file_name = "analysis/new_analysis2020.xlsx")

Number of initial observations: 562
Number of observations after applying the filter: 156


In [31]:
#visualize results for Admission
pd.read_excel("analysis/new_analysis2020.xlsx", sheet_name = "Admission AST")

,Group: Admission AST,is_woman (5),is_int (5),career_jump (5),satisfaction (5),career_service (5),mobility (8),salary (20),salary_increase_perc (5),salary_increase_abs (5),total_score,count,missing_salary_count
0,ASC,0.928571,0.059524,0.179104,0.732143,0.424603,0.988095,0.178672,0.193385,0.154855,24.839134,84,4
1,AST,0.818182,0.227273,0.132075,0.784091,0.575758,0.939394,0.180961,0.216120,0.171945,25.761582,66,3
2,DD,0.666667,0.400000,0.000000,0.833333,0.400000,0.800000,0.182251,0.261246,0.212310,23.912797,6,1
3,AST (ASTI+ASTF) + DD,0.861111,0.239437,0.118644,0.788194,0.563380,0.929577,0.181055,0.219881,0.174828,25.885106,72,4
4,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
5,General,0.974359,0.141935,0.150794,0.758013,0.488172,0.961290,0.179767,0.205614,0.164302,25.701609,156,8


In [32]:
#visualize results for Chair
chaires = pd.read_excel("analysis/new_analysis2020.xlsx", sheet_name = "Chaires")

chaires[(chaires["Group: Chaires"]=="CH ACCENTURE") |(chaires["Group: Chaires"]=="General")]

,Group: Chaires,is_woman (5),is_int (5),career_jump (5),satisfaction (5),career_service (5),mobility (8),salary (20),salary_increase_perc (5),salary_increase_abs (5),total_score,count,missing_salary_count
11,CH ACCENTURE,1.000000,0.000000,0.250000,0.833333,0.500000,1.00000,0.107326,0.170337,0.118877,24.509247,6,0
23,General,0.974359,0.141935,0.150794,0.758013,0.488172,0.96129,0.179767,0.205614,0.164302,25.701609,156,8
